# Basic plotting tools

hv resources
- Gallery: https://holoviews.org/reference/
- https://holoviews.org/user_guide/Customizing_Plots.html
- https://holoviews.org/user_guide/Style_Mapping.html
- https://holoviews.org/user_guide/Colormaps.html
- https://holoviews.org/user_guide/Plotting_with_Bokeh.html

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import holoviews as hv

import hv_anndata
from hv_anndata import ACCESSOR as A
from hv_anndata import register
from hv_anndata import scanpy as hv_sc

register()

hv.extension("bokeh")

In [ ]:
import numpy as np
import scanpy as sc

rng = np.random.default_rng()

In [ ]:
adata = sc.datasets.pbmc68k_reduced()
adata.layers["counts"] = adata.raw.X
del adata.raw
sc.tl.umap(adata)
adata

{func}`scanpy.pl.scatter`

missing features:
- `use_raw` (deprecated!)
- `na_color` (not super important)

missing convenience:
- `basis` for easy X&Y

In [ ]:
(
    hv_sc.scatter(adata, A, ["PSAP", "C1QA"], color=A.obs["bulk_labels"]).opts(
        cmap="tab10", show_legend=False
    )
    + hv_sc.scatter(
        adata, A.layers["counts"], ["PSAP", "C1QA"], color=A.obs["bulk_labels"]
    ).opts(cmap="tab10")
)

In [ ]:
# add NAs to check how missing values look
adata_scatter = adata.copy()
adata_scatter.obs.loc[
    (
        (adata_scatter.obs["bulk_labels"] == "Dendritic")
        & rng.choice([True, False], size=len(adata_scatter))
    ),
    "bulk_labels",
] = np.nan

hv_sc.umap(adata_scatter) + hv_sc.umap(adata_scatter, color=A.obs["bulk_labels"]).opts(
    cmap="tab10"
)

{func}`scanpy.pl.heatmap`

missing:
- `groupby` / [TickBar](https://github.com/holoviz/holoviews/issues/6658)
- `standard_scale` (see implemantation in {class}`hv_anndata.Dotmap`)
  - maybe done using [`dim` expressions](https://holoviews.org/user_guide/Transforming_Elements.html)?
- dendrogram doesn’t work on heatmap (again ndim problem: dendrogram should tread `(n, 1)` as 1D)

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
(
    hv_sc.heatmap(adata[:, markers], A, [A.obs["n_counts"]], add_dendrogram="obs")
    + hv_sc.heatmap(adata[:, markers], A.layers["counts"], [A.obs["n_counts"]])
).cols(1).opts(hv.opts.HeatMap(xticks=0, width=800, height=300)).opts(shared_axes=False)

TODO: support sparse data

In [ ]:
hv_sc.heatmap(adata[:40], A.obsp["distances"]).opts(tools=["hover"])

{func}`scanpy.pl.dotplot`

missing:
- `var_group_*` (highlight groups of `var_names` by drawing brackets)

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
dm = hv_anndata.Dotmap(adata=adata, marker_genes=markers, groupby="bulk_labels")
dm

In [ ]:
hv.operation.dendrogram(
    dm.plot(), adjoint_dims=["cluster"], main_dim="mean_expression", invert=True
)

{func}`scanpy.pl.tracksplot`

missing:
- `shared_xaxis=False` doesn’t seem to work on GridSpace
- GridSpace can only be 2D or row-only, not col-only
- [GridSpace `show_legend` broken](https://github.com/holoviz/holoviews/issues/5438)

In [ ]:
hv_sc.tracksplot(
    adata, ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"], color=A.obs["bulk_labels"]
).opts(hv.opts.Curve(frame_width=900))

{func}`scanpy.pl.violin`

missing:
- `density_norm`

In [ ]:
hv_sc.violin(adata, A.obs[["percent_mito", "n_counts", "n_genes"]]).opts(
    hv.opts.Violin(ylim=(0, None))
)

In [ ]:
hv_sc.violin(adata, A.obs["S_score"], color=A.obs["bulk_labels"]).opts(
    width=500, xrotation=30
)

{func}`scanpy.pl.stacked_violin`

missing:
- see tracksplot above
- can’t do `.hist()` or `.operations.dendrogram` on GridSpace
- slow!

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
hv_sc.stacked_violin(adata[:, markers], A.var.index, A.obs["bulk_labels"]).opts(
    hv.opts.Violin(frame_height=50, frame_width=50)
)

{func}`scanpy.pl.matrixplot`

missing:
- `aspect="equal"` breaks (bokeh tries to divide `None/None`)
- hist doesn’t align properly

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
hv.HeatMap(
    sc.get.aggregate(adata[:, markers], "bulk_labels", "mean"),
    [A.obs.index, A.var.index],
    [A.layers["mean"][:, :]],
).opts(xrotation=30, frame_height=180, frame_width=300).hist(num_bins=6)

{func}`scanpy.pl.clustermap`

missing:
- TickBar (see above)

In [ ]:
hv_anndata.ClusterMap(adata=adata)

{func}`scanpy.pl.dendrogram`: {class}`holoviews.operation.dendrogram`